## 내용

매주 월요일 오전 6시 기준 전주의 데이터를 축적하여 개수의 합을 구하고 summary를 제미나이를 통해서 다시 도출

### 방법
1. sector_detail/"섹터"/dates/"[해당 주차 월요일 일자]"(월-일) 불러오기 
2. summary, counts를 축적시키기
3. 축적된 summary 제미나이에게 보내서 축약하기
4. weeks에 저장


In [11]:
## 시스템 import 위치 지정
import sys
from pathlib import Path

# back 디렉토리를 sys.path에 추가
back_dir = Path(__file__).parent.parent if '__file__' in globals() else Path().resolve().parent
sys.path.append(str(back_dir))

In [12]:
## sectors 불러오기
from data.telegram_raw_datas import sectors

sectors = sectors.keys()

In [13]:
from scripts.firebase.auth import firebase_auth

db = firebase_auth()

In [14]:
from datetime import datetime, timedelta


# 해당일자의 전주 월요일을 받기 위한 함수
## input, output 값: datetime 형식의 날짜
def get_last_monday(date):
    days_to_this_monday = date.weekday()
    last_monday = date - timedelta(days=days_to_this_monday + 7)
    return last_monday

In [15]:
from scripts.gemini.ask_gemini import GeminiClient

gemini_client = GeminiClient()

In [16]:
def sum_counts(sector_details):
    total_negative = 0
    total_positive = 0
    total_neutral = 0
    for detail in sector_details:
        if detail and 'counts' in detail:
            counts = detail['counts']
            total_negative += counts.get('negative', 0)
            total_neutral += counts.get('neutral', 0)
    
    return {
        'negative': total_negative,
        'neutral': total_neutral,
        'positive': total_positive
    }

In [17]:
def sum_details(sector_details):
    total_ideas = []
    for detail in sector_details:
        if detail and 'summary' in detail:
            summary = detail['summary']
            total_ideas.append(summary)
    return str(total_ideas)


In [18]:
def get_sector_details(date):
    for sector in sectors:
        sector_details = []
        last_monday = get_last_monday(date)
        last_monday_str = last_monday.strftime("%Y-%m-%d")
        print(last_monday, sector, "시작")
        for i in range(7):
            current_date = last_monday + timedelta(days=i)
            current_date_str = current_date.strftime("%Y-%m-%d")
            sector_detail = db.collection('sector_detail').document(sector).collection('dates').document(current_date_str).get().to_dict()
            sector_details.append(sector_detail)
        week_summary = gemini_client.ask_week(sector, sum_details(sector_details))
        final = {'counts': sum_counts(sector_details), 'summary': week_summary}
        db.collection('sector_detail').document(sector).collection('weeks').document(last_monday_str).set(final)

In [19]:
def main_algoritm(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d") 
    end = datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start
    while current_date <= end:
        current_date += timedelta(days=7)
        curent_str = current_date.strftime("%Y-%m-%d")
        print(f"처리 중인 날짜: {curent_str}")
        get_sector_details(current_date)

In [22]:
# 사용
# start_date = "2025-05-30"
start_date = "2025-07-27"
# end_date = "2025-06-12"
end_date = "2025-08-23"

main_algoritm(start_date, end_date)

처리 중인 날짜: 2025-08-03
2025-07-21 00:00:00 게임 시작
2025-07-21 00:00:00 조선 시작
[시도 1] JSON 파싱 오류: Extra data: line 17 column 1 (char 945)
2025-07-21 00:00:00 방산 시작
2025-07-21 00:00:00 반도체 시작
2025-07-21 00:00:00 이차전지 시작
2025-07-21 00:00:00 디스플레이 시작
2025-07-21 00:00:00 화장품 시작
2025-07-21 00:00:00 자동차 시작
2025-07-21 00:00:00 건설 시작
[시도 1] JSON 파싱 오류: Extra data: line 17 column 1 (char 948)
2025-07-21 00:00:00 철강 시작
2025-07-21 00:00:00 화학 시작
2025-07-21 00:00:00 엔터 시작
[시도 1] JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
2025-07-21 00:00:00 음식료 시작
2025-07-21 00:00:00 패션 시작
[시도 1] JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
[시도 2] JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
[시도 3] JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
최대 재시도 횟수 초과. 실패 처리.
2025-07-21 00:00:00 풍력 시작
[시도 1] JSON 파싱 오류: Extra data: line 17 column 1 (char 863)
2025-07-21 00:00:00 원전 시작
2025-07-21 00:00:00 바이오 시작
2025-07-21 00:00:00 임플란트 시작
[시도 1] JSON 파싱 오류: Expecting value: line 1 column 1 (char